## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lebu,CL,-37.6167,-73.6500,7.80,87,0,2.90,clear sky
1,Yellowknife,CA,62.4560,-114.3525,20.29,64,75,4.63,broken clouds
2,Chokurdakh,RU,70.6333,147.9167,7.62,67,100,8.83,overcast clouds
3,Cape Town,ZA,-33.9258,18.4232,10.49,75,0,1.79,clear sky
4,Port Alfred,ZA,-33.5906,26.8910,13.07,61,89,3.32,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 20
What is the maximum temperature you would like for your vacation? 30


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Yellowknife,CA,62.4560,-114.3525,20.29,64,75,4.63,broken clouds
5,Yinchuan,CN,38.4681,106.2731,23.58,50,2,2.45,clear sky
10,Kasur,PK,31.1156,74.4467,29.06,42,1,1.92,clear sky
11,Sao Filipe,CV,14.8961,-24.4956,24.36,81,19,2.91,few clouds
12,Peniche,PT,39.3558,-9.3811,23.71,95,2,5.75,clear sky
13,Jiaozuo,CN,35.2397,113.2331,23.73,74,0,0.93,light rain
19,Suez,EG,29.9737,32.5263,25.75,62,0,3.65,clear sky
20,Hilo,US,19.7297,-155.0900,27.85,72,20,3.09,few clouds
22,Sola,VU,-13.8833,167.5500,26.37,78,90,2.57,overcast clouds
25,Vaini,TO,-21.2000,-175.2000,23.09,78,20,5.66,few clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   366
Country                366
Lat                    366
Lng                    366
Max Temp               366
Humidity               366
Cloudiness             366
Wind Speed             366
Current Description    366
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Yellowknife,CA,62.4560,-114.3525,20.29,64,75,4.63,broken clouds
5,Yinchuan,CN,38.4681,106.2731,23.58,50,2,2.45,clear sky
10,Kasur,PK,31.1156,74.4467,29.06,42,1,1.92,clear sky
11,Sao Filipe,CV,14.8961,-24.4956,24.36,81,19,2.91,few clouds
12,Peniche,PT,39.3558,-9.3811,23.71,95,2,5.75,clear sky


In [16]:
clean_travel_cities.count()

City                   366
Country                366
Lat                    366
Lng                    366
Max Temp               366
Humidity               366
Cloudiness             366
Wind Speed             366
Current Description    366
dtype: int64

In [17]:
clean_travel_cities.columns

Index(['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Current Description'],
      dtype='object')

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yellowknife,CA,20.29,broken clouds,62.4560,-114.3525,
5,Yinchuan,CN,23.58,clear sky,38.4681,106.2731,
10,Kasur,PK,29.06,clear sky,31.1156,74.4467,
11,Sao Filipe,CV,24.36,few clouds,14.8961,-24.4956,
12,Peniche,PT,23.71,clear sky,39.3558,-9.3811,
13,Jiaozuo,CN,23.73,light rain,35.2397,113.2331,
19,Suez,EG,25.75,clear sky,29.9737,32.5263,
20,Hilo,US,27.85,few clouds,19.7297,-155.0900,
22,Sola,VU,26.37,overcast clouds,-13.8833,167.5500,
25,Vaini,TO,23.09,few clouds,-21.2000,-175.2000,


In [19]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yellowknife,CA,20.29,broken clouds,62.4560,-114.3525,The Explorer Hotel
5,Yinchuan,CN,23.58,clear sky,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
10,Kasur,PK,29.06,clear sky,31.1156,74.4467,Qudrat ullah khan
11,Sao Filipe,CV,24.36,few clouds,14.8961,-24.4956,Hotel Xaguate
12,Peniche,PT,23.71,clear sky,39.3558,-9.3811,Hotel Star inn Peniche
13,Jiaozuo,CN,23.73,light rain,35.2397,113.2331,Days Hotel and Suites Jiaozuo
19,Suez,EG,25.75,clear sky,29.9737,32.5263,Green House
20,Hilo,US,27.85,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Sola,VU,26.37,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
25,Vaini,TO,23.09,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yellowknife,CA,20.29,broken clouds,62.4560,-114.3525,The Explorer Hotel
5,Yinchuan,CN,23.58,clear sky,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
10,Kasur,PK,29.06,clear sky,31.1156,74.4467,Qudrat ullah khan
11,Sao Filipe,CV,24.36,few clouds,14.8961,-24.4956,Hotel Xaguate
12,Peniche,PT,23.71,clear sky,39.3558,-9.3811,Hotel Star inn Peniche
13,Jiaozuo,CN,23.73,light rain,35.2397,113.2331,Days Hotel and Suites Jiaozuo
19,Suez,EG,25.75,clear sky,29.9737,32.5263,Green House
20,Hilo,US,27.85,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Sola,VU,26.37,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
25,Vaini,TO,23.09,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [23]:
 # 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
 # 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))